# 5分钟Python实现TXT和MySQL数据Join

## 一、实现步骤：
1. 使用pandas的接口读取数据到dataframe
2. 使用pandas的merge实现不同dataframe的join
3. 将结果数据输出到csv/excel/mysql

### 语法1：Pandas读取MySQL数据表
**read_sql(sql, con)**  
Read SQL query or database table into a DataFrame.

### 语法2：Pandas读取TXT数据
**pd.read_csv(filepath_or_buffer, sep, header)**  
Returns a DataFrame
* filepath_or_buffer：指定文件路径
* sep：字段分隔符
* header：指定列名

### 语法3：Pandas的Merge函数实现Join
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
* left, right：DataFrame
* how：join的方法分为'left', 'right', 'outer', 'inner'
* left_on, right_on：Join的Key

## 二、实战：实现csv日志文件和MySQL码表的JOIN

In [1]:
import pandas as pd

### 1、读取MySQL数据源到Dataframe

In [2]:
import pymysql
conn = pymysql.connect(
        host='127.0.0.1',
        user='root',
        password='12345678',
        database='test',
        charset='utf8'
    )

In [3]:
mysql_page = pd.read_sql("select * from tb_page", con=conn)

In [4]:
mysql_page

id name
0  1001   首页
1  1002  列表页
2  1003  详情页

### 2、读取TXT日志数据源到Dataframe

In [5]:
txt_log = pd.read_csv("./page_pvuv.txt", sep="\t", header=None)

In [6]:
txt_log.columns = ["pdate", "page_id", "pv", "uv"]

In [7]:
txt_log

pdate  page_id   pv  uv
0  2019-09-01     1001  999  99
1  2019-09-01     1002  888  88
2  2019-09-01     1003  777  77
3  2019-09-02     1001  666  66
4  2019-09-02     1002  555  55
5  2019-09-02     1003  444  44
6  2019-09-03     1001  333  33
7  2019-09-03     1002  222  22
8  2019-09-03     1003  111  11

### 3、实现两个dataframe的Join

In [8]:
merge_data = pd.merge(
    left=txt_log,
    right=mysql_page,
    how='inner',
    left_on="page_id",
    right_on="id"
)

In [9]:
merge_data

pdate  page_id   pv  uv    id name
0  2019-09-01     1001  999  99  1001   首页
1  2019-09-02     1001  666  66  1001   首页
2  2019-09-03     1001  333  33  1001   首页
3  2019-09-01     1002  888  88  1002  列表页
4  2019-09-02     1002  555  55  1002  列表页
5  2019-09-03     1002  222  22  1002  列表页
6  2019-09-01     1003  777  77  1003  详情页
7  2019-09-02     1003  444  44  1003  详情页
8  2019-09-03     1003  111  11  1003  详情页

### 4、将结果数据存储到Excel文件

In [10]:
merge_data[['pdate','page_id','name','pv','uv']].to_excel("merge_pvuv_page.xlsx")